# How it works

1. User places CSV file from the Bank in the directory
2. Program imports it into a pandas dataframe cleans it up, fills in NaNs, extracts and adds the year and month

    The program works with 2 dictionaries - the Categoriser and the Category Mapper

    * The Categoriser is a list of key words that asssign a subcategory to each entry
    * The Category Mapper assigns categories and classes to subcategories


3. The program goes through the CSV line by line, scanning for subcategory key words. Three cases follow:

* Case 1: No results found --> the user is prompted to enter a new keyword/subcategory pair into the dictionary OR make it a once-off classification
* Case 2: One result found --> the program assigns the subcategory to the keyword
* Case 3: More than one result found --> the program is in danger of making a mistaken classification because the user hasn't chosen their keyword closely enough. The user is prompted to 


# Start Up

In [2]:
import json
import os
import pandas as pd

In [3]:
%cd "F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/"

F:\Google Drive\JupyterNotebooks\Useful Projects\Budget Tracker


In [4]:
def print_dict(my_dict):
    for key,value in sorted(my_dict.items()):
        print(key,":",value)

In [5]:
def list_subcategory_options():
    with open("F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/Budget_Tracking_Category_Map.json","r") as fh:
            category_map_dict = json.load(fh)
    subcats_list = sorted(category_map_dict.keys())
    subcat_chooser = dict(zip(range(len(subcats_list)),subcats_list))
    return(subcat_chooser)

In [6]:
list_subcategory_options()

{0: 'Adobe Subscriptions',
 1: 'Anniversary',
 2: 'Antivirus',
 3: 'Audible',
 4: 'Aussielent',
 5: 'Blinkist',
 6: 'Books',
 7: 'Boost',
 8: 'CARE Australia',
 9: 'Camera Gear',
 10: 'Cash',
 11: 'Centrelink',
 12: 'Clothing',
 13: 'Comprehensive Insurance',
 14: 'Computer',
 15: 'Courses',
 16: 'Credit Card Interest',
 17: 'Dad',
 18: 'Dan',
 19: 'Department of Education',
 20: 'Desexing/Medical',
 21: 'Dog Food',
 22: 'Dog Rego',
 23: 'Drone',
 24: 'Eating Out',
 25: 'Electricity',
 26: 'Exercise Equipment',
 27: 'Festivals',
 28: 'Food',
 29: 'Fuel',
 30: 'Gifts Received',
 31: 'Google Drive',
 32: 'Greenslip',
 33: 'Home Contents',
 34: 'Interest',
 35: 'Internet',
 36: 'Jim',
 37: 'Life Insurance',
 38: 'Mel Rent',
 39: 'Misc Other Income',
 40: 'Missy',
 41: 'Movies',
 42: 'Mum',
 43: 'Museums',
 44: 'NCTM',
 45: 'NRMA Membership',
 46: 'Naked Wines',
 47: 'Netflix/Stan',
 48: 'New Scientist',
 49: 'Other Gifts',
 50: 'Other Tax Deductible Purchases',
 51: 'Phone Upgrades',
 52:

In [7]:
def list_category_options():
    with open("F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/Budget_Tracking_Category_Map.json","r") as fh:
        category_map_dict = json.load(fh)
    subcats_list = category_map_dict.keys()
    category_list = []
    for x in subcats_list:
        category_list.append(category_map_dict[x]["Category"])
    category_list = sorted(set(category_list)) 
    cat_chooser = dict(zip(range(len(category_list)),category_list))
    return(cat_chooser)

In [8]:
list_category_options()

{0: 'ATO',
 1: 'Bank',
 2: 'Car',
 3: 'Charities+Causes',
 4: 'Creativity+Learning',
 5: 'Debts+Loans',
 6: 'Department of Education',
 7: 'Dog',
 8: 'Entertainment',
 9: 'Gifts',
 10: 'Gifts Received',
 11: 'Life',
 12: 'Misc',
 13: 'Misc Other Income',
 14: 'Other Insurance',
 15: 'Rebate',
 16: 'Subscriptions',
 17: 'TFR',
 18: 'Tax Deductible Purchases',
 19: 'Technology',
 20: 'Test',
 21: 'Uncategorised',
 22: 'Utilities'}

In [9]:
def add_subcategory():
    import json
    with open("C:/Users/peter/Google Drive/JupyterNotebooks/BudgetTracking_CategoryMapping.json","r") as cats:
            cats_dict = json.load(cats)
    
    #print out the pre-existing options
    print_dict(cats_dict)
    
    new_subcat = input("What is your sub-category?")
    # testing to see if it already exists
    while new_subcat in cats_dict:
# room for checking if this is in or NEARLY in the thing already    
        new_subcat = input("Oops. That category already extists. Choose another. ")
    # once we have a unique value, enter the category and class
    new_cat = input("What is your category?")
    new_class = input("What is your class?")
    
    #now actually adding the new category to the repository
    cats_dict[new_subcat] = {"Category":new_cat,"Class":new_class}
    #save the file
    print("Adding new subcategory:",new_subcat)
    with open("C:/Users/peter/Google Drive/JupyterNotebooks/BudgetTracking_CategoryMapping.json","w") as savefile:
            cats_dict = json.dump(cats_dict,savefile,indent=2, sort_keys = True)

In [10]:
def pd_add_categories(eachSubcategory):
    # open the categories pane
    with open("F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/Budget_Tracking_Category_Map.json","r") as fh:
        categories = json.load(fh)
    
    if eachSubcategory in categories:
        return(cats[eachSubcategory]["Category"])
    
    else:
        return("Uncategorised")

In [11]:
def pd_add_classes(eachSubcategory):
    with open("F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/Budget_Tracking_Category_Map.json","r") as openfile:
        cats = json.load(openfile)
        
    if eachSubcategory in cats:
        return(cats[eachSubcategory]["Class"])
    
    else:
        return("Uncategorised")

In [12]:
def get_month(eachDate):
    return(eachDate.month)

In [13]:
def get_year(eachDate):
    return(eachDate.year)

In [14]:
def save_to_excel(df,filename,sheetname):
    path_name = "F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/"+filename+".xlsx"
    print("Saving to",path_name+"?")
    if input("Continue? Y or N") == "Y":
# insert check for overwrite of file here
        df.to_excel(path_name,sheet_name=sheetname)
    elif "N":
        print("Save cancelled")

In [ ]:
def assign_subcategory(eachDescription):
    
    match_dict = kw_match_tracker(eachDescription)["match_dict"]
    hit_count = kw_match_tracker(eachDescription)["hit_count"]                                                            #"{} --> {}".format(eachKey,keyword_map[eachKey])
            
    if hit_count == 0:
        print("No Matches found for '",eachDescription,"'")
        user_choice = str.upper(input("Type N for new keyword, C to categorise as a once-off or S for skip"))
        if user_choice == "N":
            return(new_keyword())
            
        elif user_choice == "C":
            print_dict(list_subcategory_options())
            subcat = int(input("Make your choice from the list above by entering the number"))
            return(list_subcategory_options()[subcat])
        
        elif user_choice == "S":
            return("Uncategorised")
    
    elif hit_count == 1:
        return(keyword_map[match_dict[hit_count]])
    
    elif hit_count > 1:
        print("More than one hit found for '{}'".format(eachDescription))
        print_dict(match_dict)
        shortlist_choice = int(input("Please choose a number from the options above or press enter to skip."))
        
        return(keyword_map[match_dict[shortlist_choice]])

In [ ]:
desc = "TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savings Savings"
assign_subcategory(desc)

In [18]:
def kw_match_tracker(eachDescription):
    path = "F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/AutoCatV2.json"
    with open(path,"r") as keyword_map:
        keyword_map = json.load(keyword_map)
        
    keyword_list = list(keyword_map.keys())
    hit_count = 0
    
    match_dict = {}
    for eachKey in keyword_list:
        #print("searching for {} in {}".format(eachKey,eachDescription))
        if eachKey in eachDescription:
            hit_count += 1
            #print("Hit count: ",str(hit_count))
            match_dict[hit_count] = eachKey
    
    output = {"match_dict":match_dict,"hit_count":hit_count}
    return(output)

In [20]:
desc = "TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savings Savings"
kw_match_tracker(desc)

{'hit_count': 2, 'match_dict': {1: '224238S6', 2: 'TFR'}}

In [52]:
def new_keyword():
    path = "F:/Google Drive/JupyterNotebooks/Useful Projects/Budget Tracker/AutoCatV2.json"
    with open(path,"r") as fh:
        keyword_map = json.load(fh)
    
    new_kw = input("Please choose a keyword: ")
    print("New keyword: {} ".format(new_kw))
    options = list_subcategory_options()
    
    print_dict(options)
    
    subcat_choice = int(input("Enter the number of the subcategory mapping you want to use: "))
    subcat = options[subcat_choice]
    keyword_map[new_kw]  = subcat
    
    save_option = str.upper(input("Save new keyword mapping {} : {}? Press Y ".format(new_kw,subcat)))
    
    if save_option == "Y":
        with open(path,"w") as fh:
            json.dump(keyword_map,fh,sort_keys=True,indent=2)
    
    return(subcat)

In [53]:
new_keyword()

Please choose a keyword: Teachers Feder
New keyword: Teachers Feder 
0 : Adobe Subscriptions
1 : Anniversary
2 : Antivirus
3 : Audible
4 : Aussielent
5 : Blinkist
6 : Books
7 : Boost
8 : CARE Australia
9 : Camera Gear
10 : Cash
11 : Centrelink
12 : Clothing
13 : Comprehensive Insurance
14 : Computer
15 : Courses
16 : Credit Card Interest
17 : Dad
18 : Dan
19 : Department of Education
20 : Desexing/Medical
21 : Dog Food
22 : Dog Rego
23 : Drone
24 : Eating Out
25 : Electricity
26 : Exercise Equipment
27 : Festivals
28 : Food
29 : Fuel
30 : Gifts Received
31 : Google Drive
32 : Greenslip
33 : Home Contents
34 : Interest
35 : Internet
36 : Jim
37 : Life Insurance
38 : Mel Rent
39 : Misc Other Income
40 : Missy
41 : Movies
42 : Mum
43 : Museums
44 : NCTM
45 : NRMA Membership
46 : Naked Wines
47 : Netflix/Stan
48 : New Scientist
49 : Other Gifts
50 : Other Tax Deductible Purchases
51 : Phone Upgrades
52 : Radiotopia
53 : Rebate
54 : Rego
55 : Repairs
56 : Retention Bonus
57 : Savings
58 : S

"Teacher's Fed"

In [47]:
def clean_data(df):
    df.fillna("NA",inplace = True)

# Testing Area

In [ ]:
os.listdir()

In [51]:
df= pd.read_excel("March April 2015.xlsx")
clean_data(df)
#df.fillna("NA",inplace=True)
df["Subcategory"] = df.apply(lambda x: assign_subcategory(df["Description"]),axis= 1)

No Matches found for ' 0     TFR TO 484799 502175908 ONLINE To-MISS MELISSA...
1     TFR TO 032001 143827 ONLINE To-MUNGINDI CENTRA...
2     TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...
3     DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...
4                   TF From: TF Ref: NSW Teachers Feder
5     TFR TO 932000 652932 ONLINE To-MR PETER NEWMAN...
6     TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savin...
7     TFR TO 932000 652932 MOB To-MR PETER NEWMAN Re...
8     DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...
9     TFR TO 484799 502175908 ONLINE To-MISS MELISSA...
10    TFR to 224238S6 ONLINE To-P W NEWMAN Ref-Savin...
11                  TF From: TF Ref: NSW Teachers Feder
12    DIRECT DEBIT From: TPG Internet Ref: DF3Q0PJTS...
13            BPAY BOSTES ONLINE Ref-2763787 #134329061
14    TELSTRAKENAN20EASYPAYA From: Telstra DDebit Re...
15    DIRECT CREDIT From: DEPT OF SCHOOL E Ref: DOSE...
16                  TF From: TF Ref: NSW Teachers Feder
17    TFR TO 932000 65293

NameError: ("name 'new_keyword' is not defined", 'occurred at index 1')